# The <code>cupy</code> Package

Multiprocessing is a topic in this course, the focus of which was intended to be multiprocessing with the CPU.  Given the advances in the <code>cupy</code> package recently, a quick tangent to cover this method of multiprocessing on the Graphic Processing Units (GPUs) is worthwhile because some computers are equipped with powerful graphics cards with a multitude of onboard processors.  Although each of these processors is much less powerful than the cores on our CPUs, their power lies in their multitude.  Here are the specs on my NVIDIA GeForce RTX 3090 graphics card:

- 10496 computing cores
- 24GB memory

There is strength in the number of processors.  The GPU memory capacity, 24GB, is less than I have available to my CPU (64GB) and so this is a relative weakness.

NVIDA's <code>cuda</code> is used by the <code>cupy</code> package and so only cuda-enabled NVIDIA graphcis card can run <code>cupy</code>: a list of those can be found [here](https://developer.nvidia.com/cuda-gpus).  You can check to see if your graphics card is listed here.  If it is, then you can run <code>cupy</code>.

The programming interface for <code>cupy</code> currently looks just like the <code>numpy</code> interface.  To use <code>cupy</code> use this import statement,

``` python
import cupy as cp
```

and then use the <code>cp</code> alias as you would <code>np</code>.

for example see some of the commands below.

# Installing <code>cupy</code>

I recommend creating a new anaconda environment for <code>cupy</code> by following these steps:

- Open an Anaconda prompt as administrator
- Execute the command <code>conda create --name __env_name__ cupy anaconda</code> where __env_name__ is the name you want to give this new environment.  Since its feature is <code>cupy</code> that is what I would call it.  The <code>cupy</code> argument causes <code>cupy</code> and the <code>anaconda</code> argument cause hundreds of the default anaconda packages to be downloaded.
- Follow the default prompts to download and install all the packages.

# Let's Get to Work!

In [1]:
import cupy as cp

C:\Users\jrbrad\AppData\Local\anaconda3\envs\cupy\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [ ]:
cp.arange(4).reshape(2,2)

In [ ]:
cp.zeros((3,3)).astype(cp.float32)

The advantage of <code>cupy</code> is that it used the many GPU processors and the variable data are structured so that computations can be distributed among the GPU processors and portions of a computation carried out in parallel fashion.